In [782]:
%reset -f

In [783]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [784]:
# This script is to process from the ADC Filter Onwards assuming that other steps are complete

In [785]:
import os
import sys
sys.path.append(os.path.abspath('../../Code/Software/CarDpy-master'))
from cardpy.Sample_Data.Data_Paths                  import *
from cardpy.Data_Import                             import *
from cardpy.Data_Sorting                            import *
from cardpy.Data_Saving                             import *
from cardpy.Data_Processing.Gibbs                   import *
from cardpy.Data_Processing.Registration            import *
from cardpy.Data_Processing.Rejection               import *
from cardpy.Data_Processing.Respiratory             import *
from cardpy.Data_Processing.Diffusivity             import *
from cardpy.Data_Processing.Averaging               import *
from cardpy.Data_Processing.Denoising               import *
from cardpy.Data_Processing.Interpolation           import *
from cardpy.Data_Processing.Segmentation_Matrix_DTI import *

from cardpy.GUI_Tools                     import *
from cardpy.Colormaps                     import *
# import cardpy

In [786]:
data_type                 = 'NifTis'         # Input data type for sample Data (DICOM or NifTis)

### User Settings
DICOM_reader_info         = 'ON'             # Flag to read from DICOM folder
operation_type            = 'Magnitude'      #

Gibbs_Mode                = 'ON'             # Flag to register data

Rejection_Mode            = 'ON'             # Flag to reject and replace bad acquisitions
NRMSE_Threshold           = 0.75
NSSIM_Threshold           = 0.75 
rejection_zoom            = 'ON'
rejection_IntEACT_zoom    = 'ON'
rejection_organ           = 'Heart'

Respiratory_Mode          = 'OFF'             # Flag to reject and replace bad acquisitions
respiratory_zoom          = 'ON'
respiratory_IntERACT_zoom = 'ON'
respiratory_organ         = 'Stomach'

Registration_Mode         = 'ON'             # Flag to register data
registration_algorithm    = 'Affine'         # Type of registration to perform
temporary_denoising       = 'OFF'

ADC_Filter_Mode          = 'ON'
Averaging_Mode           = 'ON'             # Flag to average data

Denoising_Mode           = 'OFF'             # Flag to average data
number_of_coils          = 20
denoising_algorithm      = 'LocalPCA'       # Type of denoising to perform

Interpolation_Mode       = 'OFF'             # Flag to interpolate data

Extended_Matrix_Mode     = 'ON'             # Flag to interpolate data

E1_Mode                  = 'ON'


if registration_algorithm == 'Affine':
    sub_registration = 'Rigid'
if registration_algorithm == 'Rigid':
    sub_registration = 'Translation'
if registration_algorithm == 'Translation':
    sub_registration = 'Translation'

In [787]:
### Load in Registered Data
### Change voxel resolution and volunteer for each 
volunteer = '/V011'
res = '/vol_2.0res_8.0sl'

if data_type == 'DICOM':
    DICOM_Folder = os.path.abspath('../../Data/Dicoms/'+volunteer+res+'/')
    print(DICOM_Folder)
    [matrix_stacked, b_vals_stacked, b_vecs_stacked, Header] = DICOM_Reader(DICOM_Folder, info='ON')
if data_type == 'NifTis':
    main_path = os.path.abspath('../../Data/NifTi/' + volunteer+res+'/03_Registered/')
    NifTi_path     = os.path.join(main_path, 'Registered.nii')
    b_values_path  = os.path.join(main_path, 'Registered.bvals')
    b_vectors_path = os.path.join(main_path, 'Registered.bvecs')
    header_path    = os.path.join(main_path, 'Registered.header')
    
    print(NifTi_path)
    print(b_values_path)
    print(b_vectors_path)
    print(header_path)
    
    [matrix_stacked, b_vals_stacked, b_vecs_stacked, Header, _, _] = NifTi_Reader(NifTi_path, b_values_path, b_vectors_path, header_path)

temporary_matrix_stacked = matrix_stacked
temporary_bvals_stacked  = b_vals_stacked
temporary_bvecs_stacked  = b_vecs_stacked
out_path                 = os.path.abspath('../../Data/NifTi'+volunteer+res+'/ADC_Filtered')

### In order to use this script with your own data, 
### please define the variables in the print statments of this cell.
### - DICOM_Folder for DICOMs
### - NifTi_path, b_values_path, b_vectors_path, and header_path for NifTis



/Volumes/T7/Voxel_Bank/Data/NifTi/V011/vol_2.0res_8.0sl/03_Registered/Registered.nii
/Volumes/T7/Voxel_Bank/Data/NifTi/V011/vol_2.0res_8.0sl/03_Registered/Registered.bvals
/Volumes/T7/Voxel_Bank/Data/NifTi/V011/vol_2.0res_8.0sl/03_Registered/Registered.bvecs
/Volumes/T7/Voxel_Bank/Data/NifTi/V011/vol_2.0res_8.0sl/03_Registered/Registered.header


In [788]:
########## Configuration Module #################################################################################################################
counter        = 3
[temporary_matrix_sorted,  temporary_bvals_sorted,  temporary_bvecs_sorted]          = stacked2sorted(temporary_matrix_stacked, 
                                                                                                      temporary_bvals_stacked, 
                                                                                                      temporary_bvecs_stacked)

################################################################################################################################################
########## ADC Filter Module ###################################################################################################################
if ADC_Filter_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Diffusivity_Filtered'
    print('ADC Filter mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted] = ADC_Filter(temporary_matrix_sorted, 
                                                                                        temporary_bvals_sorted, 
                                                                                        temporary_bvecs_sorted,
                                                                                        operation_type = operation_type)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'ADC_Filtered', Header, filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = filtered_matrix_sorted
    temporary_bvals_sorted  = filtered_bvals_sorted
    temporary_bvecs_sorted  = filtered_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted 
if Averaging_Mode == 'OFF':
    print('Averaging mode is off.')

########## Averaging Module ####################################################################################################################
if Averaging_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Averaged'
    print('Averaging mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [averaged_matrix_sorted, averaged_bvals_sorted, averaged_bvecs_sorted] = average(temporary_matrix_sorted, 
                                                                                     temporary_bvals_sorted, 
                                                                                     temporary_bvecs_sorted,
                                                                                     operation_type = operation_type)
    print('Re-registering data after data averaging.')
    ### Step 2: Perform registration on average sorted matrix
    [registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted] = register(averaged_matrix_sorted,
                                                                                            averaged_bvals_sorted, 
                                                                                            averaged_bvecs_sorted, 
                                                                                            registration_algorithm = registration_algorithm,
                                                                                            temporary_denoising    = temporary_denoising,
                                                                                            operation_type         = operation_type)
    ### Step 3: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Averaged', Header, registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted)
    ### Step 4: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = registered_matrix_sorted
    temporary_bvals_sorted  = registered_bvals_sorted
    temporary_bvecs_sorted  = registered_bvecs_sorted
    ### Step 5: Delete Variables
    del counter_string, folder, output_path
    del averaged_matrix_sorted, averaged_bvals_sorted, averaged_bvecs_sorted 
    del registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted
if Averaging_Mode == 'OFF':
    print('Averaging mode is off.')
################################################################################################################################################
########## Denoising Module ####################################################################################################################
if Denoising_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Denoised'
    print('Denoising mode is on.')
    ### Step 1: Perform denoising on temporary stacked matrix
    [denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted] = denoise(temporary_matrix_sorted, 
                                                                                     temporary_bvals_sorted, 
                                                                                     temporary_bvecs_sorted,
                                                                                     denoising_algorithm = denoising_algorithm,
                                                                                     operation_type      = operation_type)
    ### Step 2: Save denoised stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Denoised', Header, denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = denoised_matrix_sorted
    temporary_bvals_sorted  = denoised_bvals_sorted
    temporary_bvecs_sorted  = denoised_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted
if Denoising_Mode == 'OFF':
    print('Denoising mode is off.')
################################################################################################################################################
########## Interpolation Module ################################################################################################################
if Interpolation_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Interpolated'
    print('Interpolating mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted] = zero_filled(temporary_matrix_sorted, 
                                                                                                     temporary_bvals_sorted, 
                                                                                                     temporary_bvecs_sorted,
                                                                                                     operation_type = operation_type)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Header['X Resolution'] = Header['X Resolution'] / 2
    Header['Y Resolution'] = Header['Y Resolution'] / 2    
    Save_Diffusion_Image_Data(output_path, 'Interpolated', Header, interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = interpolated_matrix_sorted
    temporary_bvals_sorted  = interpolated_bvals_sorted
    temporary_bvecs_sorted  = interpolated_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted
if Interpolation_Mode == 'OFF':
    print('Interpolating mode is off.')
################################################################################################################################################
########## Extended Matrix Module ##############################################################################################################
if Extended_Matrix_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Extended_Matrix'
    print('Extended matrix mode is on.')
    ### Step 1: Perform 
    [segmentation_matrix, segmentation_bvals, segmentation_bvecs] = DTI_segmentation_matrix(temporary_matrix_sorted, 
                                                                                            temporary_bvals_sorted, 
                                                                                            temporary_bvecs_sorted)
    [segmentation_matrix, segmentation_bvals, segmentation_bvecs] = stacked2sorted(segmentation_matrix, 
                                                                                   segmentation_bvals, 
                                                                                   segmentation_bvecs)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Extended_Matrix', Header, segmentation_matrix, segmentation_bvals, segmentation_bvecs)
    ### Step 3: Delete Variables
    del counter_string, folder, output_path
    del segmentation_matrix, segmentation_bvals, segmentation_bvecs
if Extended_Matrix_Mode == 'OFF':
    print('Extended matrix mode is off.')
################################################################################################################################################
########## Primary Eigenvector Map Module ######################################################################################################
if E1_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Primary_Eigenvector'
    output_path    = os.path.join(out_path, folder)
    print('Primary Eigenvector mode is on.')
    Save_Primary_Eigenvector_Data(output_path, Header, temporary_matrix_sorted, temporary_bvals_sorted, temporary_bvecs_sorted)
if E1_Mode == 'OFF':
    print('Primary eigenvector mode is off.')
################################################################################################################################################

ADC Filter mode is on.
Low b value, nothing computed
Averaging mode is on.
Re-registering data after data averaging.
Temporary denoising is turned off.
Registering diffusion directions to first diffusion direction (b = 0) for each average.
Registering first diffusion direction (b = 0) to each average first diffusion direction (b = 0).
Finished registration.
Denoising mode is off.
Interpolating mode is off.
Extended matrix mode is on.
Primary Eigenvector mode is on.


In [789]:
temporary_matrix_sorted.shape

(60, 60, 1, 13, 1)